# 🗂️ Export Transcripts to Obsidian
Convert CSV transcripts to Markdown notes with Media Extended timestamp links, embedded validation snippets, and a quantitative questionnaire.

Workflow:
1. Load transcripts (CSV) and optional validation_report.json.
2. Create Obsidian-friendly Markdown with timestamp links to audio.
3. Append a removable validation block (italic tag) after each transcript.
4. Append the quantitative questionnaire at the end of each note.

In [5]:
# 1) Paths and inputs
from pathlib import Path
import pandas as pd
import json
import yaml
import glob
import datetime

project_root = Path('..').resolve()
config_path = project_root / 'config.yml'

# Load project config (used for default transcript/audio paths)
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

paths_cfg = config.get('paths', {})

# Directories
TRANSCRIPTS_DIR = Path(paths_cfg.get('output_base_folder', project_root)).expanduser().resolve()

# Obsidian vault setup
obsidian_vault = Path(paths_cfg.get('obsidian_vault_folder', project_root)).expanduser().resolve()
compact_audio_subfolder = paths_cfg.get('compact_audio_folder', 'compact_audio')

# Build the full audio vault path
AUDIO_VAULT_DIR = obsidian_vault / compact_audio_subfolder

# Output markdown goes to obsidian vault root
OUTPUT_DIR = obsidian_vault
VALIDATION_REPORT_PATH = project_root / 'validation_report.json'

print(f'📂 Transcripts:       {TRANSCRIPTS_DIR}')
print(f'📚 Obsidian Vault:    {obsidian_vault}')
print(f'🎧 Audio subfolder:   "{compact_audio_subfolder}"')
print(f'🎧 Audio vault path:  {AUDIO_VAULT_DIR}')
print(f'   ↳ Exists: {AUDIO_VAULT_DIR.exists()}')
if AUDIO_VAULT_DIR.exists():
    audio_count = len(list(AUDIO_VAULT_DIR.glob("*")))
    print(f'   ↳ Files found: {audio_count}')
print(f'📝 Output MD files:   {OUTPUT_DIR}')
print(f'✅ Validation report: {VALIDATION_REPORT_PATH.exists()}')


📂 Transcripts:       D:\legendary2\transcribe
📚 Obsidian Vault:    D:\legendary2\obsidian_vault
🎧 Audio subfolder:   "compact_audio"
🎧 Audio vault path:  D:\legendary2\obsidian_vault\compact_audio
   ↳ Exists: True
   ↳ Files found: 88
📝 Output MD files:   D:\legendary2\obsidian_vault
✅ Validation report: True


In [6]:
# 2) Questionnaire + settings
TARGET_SPEAKER = 'SPEAKER_00'  # bolded in output

QUESTION_SECTIONS = {
    'Section Q2: Demographics': [
        'Q2_1_person_interviewed', 'Q2_2_birthyear', 'Q2_3_residents',
        'Q2_4_year_aqcuisition', 'Q2_5_income_outside_farm'
    ],
    'Section Q3: Farm Structure': [
        'Q3_1_farmsize', 'Q3_2_leased_area', 'Q3_3_leased_out_area',
        'Q3_4_ownership_of_other_farms', 'Q3_5_started_farming',
        'Q3_6_ownership_motivation', 'Q3_7_organic', 'Q3_8_local_farming_conditions'
    ],
    'Section Q4: Crops & Livestock': [
        'Q4_1_crop_types', 'Q4_2_crop_reason', 'Q4_3_livestock',
        'Q4_4_livestock_type_count', 'Q4_5_livestock_changes_past',
        'Q4_6_livestock_changes_future'
    ],
    'Section Q5: Land Use Changes': [
        'Q5_1_arable_to_grass', 'Q5_2_grass_to_arable', 'Q5_3_arable_to_nature',
        'Q5_4_nature_to_arable', 'Q5_5_grass_to_nature', 'Q5_6_nature_to_grass',
        'Q5_7_other_changes', 'Q5_8_crop_changes_past', 'Q5_9_birds',
        'Q5_10_insects', 'Q5_11_other_animals'
    ],
    'Section Q6: Decision Making': [
        'Q6_1_planning_crop_rotation', 'Q6_2_considerations_crops',
        'Q6_3_crop_changes_future', 'Q6_4_soil_variation_crops',
        'Q6_5_soil_impact_cropchoice', 'Q6_6_terrain_impact_cropchoice',
        'Q6_7_drainage_impact_cropchoice', 'Q6_8_changes_fieldplanning',
        'Q6_9_good_agricultural_year'
    ],
    'Section Q7: Legumes': [
        'Q7_1_1_Legumes_crop_rotation', 'Q7_1_2_Challenges_legumes',
        'Q7_1_3_pros_cons_legumes', 'Q7_1_4_perception_legumes',
        'Q7_1_5_Perception_changes_legumes', 'Q7_1_6_advantages_disadvantages_legumes',
        'Q7_1_7_barriers_legumes', 'Q7_1_8_benefits_legumes', 'Q7_1_9_cooperation_sales',
        'Q7_2_1_experience_legumes', 'Q7_2_2_perception_legumes',
        'Q7_2_3_pros_cons_legumes', 'Q7_2_4_advantages_disadvantages_legumes',
        'Q7_2_5_expected_challenges_legumes', 'Q7_2_6_incentives_legumes',
        'Q7_2_7_benefits_legumes'
    ],
    'Section Q8: Field Conditions': [
        'Q8_1_best_worst_fields', 'Q8_2_drainage_system', 'Q8_3_waterlogged_fields',
        'Q8_4_drought_fields', 'Q8_5_plantprotection_fields',
        'Q8_6_plantprotection_use_change', 'Q8_7_production_conditions_fields',
        'Q8_8_production_security_fields', 'Q8_9_yield_data'
    ],
    'Section Q9: Society & Admin': [
        'Q9_1_farmtype', 'Q9_2_admin_factors_cropchoice', 'Q9_3_env_impact_choice',
        'Q9_4_societal_expectations', 'Q9_5_changes_societal_expectations',
        'Q9_6_traditions_crops', 'Q9_7_knowlegde_info_sources'
    ],
    'Section Q10: Landscape': [
        'Q10_1_landscape_role', 'Q10_2_landscape_role_changes', 'Q10_3_landscape_steward'
    ],
    'Section Q11: Future': [
        'Q11_1_future_farm_wishes', 'Q11_2_future_legumes', 'Q11_3_biodiversity_considerations',
        'Q11_4_climate_adaptation', 'Q11_5_nitrogen_management', 'Q11_6_retrospective_farm',
        'Q11_7_Green_tripartite'
    ]
}


In [7]:
# 3) Helper functions
def format_ts(seconds: float) -> str:
    seconds = int(round(seconds))
    h, m = divmod(seconds, 3600)
    m, s = divmod(m, 60)
    return f"{h:02}:{m:02}:{s:02}"

def find_audio_file(base_id: str, audio_dir: Path) -> str | None:
    """Find audio file matching CSV basename.
    
    Flexible matching: Interview_1 matches interview_1.mp3, interview 1.mp3, etc.
    """
    if not audio_dir.exists():
        return None
    
    import re
    
    # Get all audio files
    all_audio = [f for f in audio_dir.glob("*") if f.is_file()]
    
    # Normalize the base_id: convert to lowercase and extract any numbers
    base_lower = base_id.lower()
    
    # Extract numeric ID if present (e.g., "interview_1" -> "1")
    num_match = re.search(r'\d+', base_id)
    numeric_id = num_match.group(0) if num_match else None
    
    # Strategy 1: Exact stem match (case-insensitive)
    for audio in all_audio:
        if audio.stem.lower() == base_lower:
            return audio.name
    
    # Strategy 2: Flexible word + number matching
    # Split base into words and numbers, then match flexibly
    if numeric_id:
        for audio in all_audio:
            audio_lower = audio.stem.lower()
            # Check if numeric ID is in the audio filename
            if numeric_id in audio_lower:
                # Also check if common prefixes match
                if any(prefix in audio_lower for prefix in ['interview', base_lower.split('_')[0] if '_' in base_lower else '']):
                    return audio.name
    
    # Strategy 3: Simple substring match
    for audio in all_audio:
        if base_lower in audio.stem.lower():
            return audio.name
        if audio.stem.lower() in base_lower:
            return audio.name
    
    # Strategy 4: Numeric-only match (if base has a number)
    if numeric_id:
        for audio in all_audio:
            if numeric_id in audio.stem:
                return audio.name
    
    return None

def load_validation_map(report_path: Path) -> dict:
    if not report_path.exists():
        return {}
    with open(report_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    index = {}
    for item in data.get('results', []):
        name = Path(item.get('csv_file', '')).name
        if name:
            index[name] = item
    return index

def render_validation_block(csv_name: str, report: dict) -> list[str]:
    if not report:
        return []
    flagged = report.get('flagged_count', len(report.get('flagged_segments', [])))
    checked = report.get('segments_checked', 0)
    lines = []
    lines.append('> _VALIDATION_REPORT (delete if not needed)_')
    lines.append(f'> File: {csv_name}')
    lines.append(f'> Flagged: {flagged}/{checked} segments')
    for seg in report.get('flagged_segments', []):
        start = seg.get('start', 0)
        end = seg.get('end', 0)
        speaker = seg.get('speaker', 'Unknown')
        issues = seg.get('issues', [])
        issues_str = '; '.join([str(i) for i in issues]) if issues else ''
        lines.append(f'> - [{format_ts(start)}-{format_ts(end)}] {speaker}: {issues_str}')
    return lines


In [8]:
# 4) Debug: Detailed audio file matching
print("\n🔍 DETAILED AUDIO FILE MATCHING DEBUG\n")
print(f"Audio vault: {AUDIO_VAULT_DIR}")
print(f"Exists: {AUDIO_VAULT_DIR.exists()}\n")

if AUDIO_VAULT_DIR.exists():
    all_audio = sorted([f for f in AUDIO_VAULT_DIR.glob("*") if f.is_file()])
    print(f"Audio files found ({len(all_audio)}):")
    for af in all_audio[:20]:
        print(f"  • {af.name} (stem: {af.stem})")
    
    # Get sample CSVs
    csv_files_sample = sorted(TRANSCRIPTS_DIR.glob('*.csv'))[:3]
    
    print(f"\n\nTesting matching for first {len(csv_files_sample)} CSVs:\n")
    for csv in csv_files_sample:
        base = csv.stem
        print(f"CSV: {csv.name}")
        print(f"  Stem: '{base}'")
        print(f"  Searching for audio files that match...")
        
        # Try different strategies
        found = None
        
        # Strategy 1: Direct match (case-insensitive)
        for audio in all_audio:
            if audio.stem.lower() == base.lower():
                print(f"    ✅ Direct match: {audio.name}")
                found = audio.name
                break
        
        if not found:
            # Strategy 2: Substring match
            for audio in all_audio:
                if base.lower() in audio.stem.lower() or audio.stem.lower() in base.lower():
                    print(f"    ✅ Substring match: {audio.name}")
                    found = audio.name
                    break
        
        if not found:
            # Show what would match with glob
            for pattern in [f"{base}*", f"*{base}*"]:
                matches = list(AUDIO_VAULT_DIR.glob(pattern))
                if matches:
                    print(f"    📌 Glob pattern '{pattern}' would match:")
                    for m in matches:
                        print(f"       → {m.name}")
                    found = matches[0].name
                    break
        
        if not found:
            print(f"    ❌ NO MATCH FOUND")
        print()
else:
    print(f"❌ Audio vault directory does not exist!")



🔍 DETAILED AUDIO FILE MATCHING DEBUG

Audio vault: D:\legendary2\obsidian_vault\compact_audio
Exists: True

Audio files found (88):
  • interview_1.mp3 (stem: interview_1)
  • interview_10.mp3 (stem: interview_10)
  • interview_102.mp3 (stem: interview_102)
  • interview_104.mp3 (stem: interview_104)
  • interview_108.mp3 (stem: interview_108)
  • interview_110.mp3 (stem: interview_110)
  • interview_111.mp3 (stem: interview_111)
  • interview_112.mp3 (stem: interview_112)
  • interview_114.mp3 (stem: interview_114)
  • interview_12.mp3 (stem: interview_12)
  • interview_120.mp3 (stem: interview_120)
  • interview_121.mp3 (stem: interview_121)
  • interview_122.mp3 (stem: interview_122)
  • interview_126.mp3 (stem: interview_126)
  • interview_128.mp3 (stem: interview_128)
  • interview_132.mp3 (stem: interview_132)
  • interview_135.mp3 (stem: interview_135)
  • interview_136.mp3 (stem: interview_136)
  • interview_137.mp3 (stem: interview_137)
  • interview_138.mp3 (stem: interview_

In [10]:
# 5) Export CSV transcripts to Obsidian Markdown
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
AUDIO_VAULT_DIR.mkdir(parents=True, exist_ok=True)

validation_map = load_validation_map(VALIDATION_REPORT_PATH)

csv_files = sorted(TRANSCRIPTS_DIR.glob('*.csv'))
print(f'\n📝 EXPORTING {len(csv_files)} TRANSCRIPTS\n')

success_count = 0
for csv_path in csv_files:
    df = pd.read_csv(csv_path)
    base = csv_path.stem
    title = base.replace('_', ' ').title()
    out_path = OUTPUT_DIR / f'{base}.md'

    # Find audio file
    audio_name = find_audio_file(base, AUDIO_VAULT_DIR)
    audio_rel_path = f'{compact_audio_subfolder}/{audio_name}' if audio_name else None

    # Get validation data for this file
    validation_data = validation_map.get(csv_path.name, {})
    flagged_segments = {seg['segment_index']: seg for seg in validation_data.get('flagged_segments', [])}

    lines = []
    # Front matter
    lines.append('---')
    lines.append(f'title: "{title}"')
    lines.append(f'date: {datetime.date.today().isoformat()}')
    lines.append('tags: [interview]')
    lines.append('---\n')

    # Audio embedding
    if audio_rel_path:
        lines.append('# 🎧 Recording')
        lines.append(f'![[{audio_rel_path}]]\n')
        lines.append('---\n')
    
    lines.append('# Transcript\n')

    # Transcript segments with inline validation
    for idx, row in df.iterrows():
        speaker = str(row.get('Speaker', 'Unknown'))
        text = str(row.get('Text', ''))
        start = float(row.get('Start', 0))
        ts_label = format_ts(start)
        ts_int = int(round(start))
        
        # Build timestamp link
        if audio_rel_path:
            ts_link = f'[[{audio_rel_path}#t={ts_int}|{ts_label}]]'
        else:
            ts_link = f'`[{ts_label}]`'
        
        # Bold target speaker
        heading = f'### **{speaker}** {ts_link}' if speaker == TARGET_SPEAKER else f'### {speaker} {ts_link}'
        lines.append(heading)
        lines.append(text)
        
        # Add validation block if this segment has issues
        if idx in flagged_segments:
            val_seg = flagged_segments[idx]
            lines.append('')
            lines.append('> [!warning]- _Validation Issues (AI-detected, may contain false positives)_')
            
            # Handle issues - can be list of strings or list of dicts
            issues = val_seg.get('issues', [])
            if issues:
                for issue in issues:
                    if isinstance(issue, dict):
                        # Issue is a dict with type/description
                        issue_text = issue.get('description', str(issue))
                    else:
                        # Issue is a string
                        issue_text = str(issue)
                    lines.append(f'> - *#Issue:* _{issue_text}_')
            
            # Add suggestions
            suggestions = val_seg.get('suggestions')
            if suggestions:
                lines.append(f'> - *#Suggestions:* _{suggestions}_')
            
            # Add confidence
            confidence = val_seg.get('confidence')
            if confidence:
                lines.append(f'> - *Confidence:* {confidence:.0%}')
        
        lines.append('')

    # Questionnaire section
    lines.append('---')
    lines.append('# 📊 Interview Data')
    lines.append('> [!INFO]- Questionnaire')
    lines.append('> Click to expand and fill in answers.')
    lines.append('>')
    for section, questions in QUESTION_SECTIONS.items():
        lines.append(f'> %% {section} %%')
        for q in questions:
            lines.append(f'> **{q}**:: ')
        lines.append('>')

    # Write file
    out_path.write_text('\n'.join(lines), encoding='utf-8')
    
    audio_status = '🎧' if audio_rel_path else '⚠️ '
    val_count = len(flagged_segments)
    val_status = f'✅({val_count})' if val_count > 0 else '  '
    print(f'{audio_status} {val_status} {out_path.name}')
    success_count += 1

print(f'\n✨ Exported {success_count}/{len(csv_files)} files to {OUTPUT_DIR}')



📝 EXPORTING 88 TRANSCRIPTS

🎧 ✅(7) Interview_1.md
🎧 ✅(11) Interview_10.md
🎧 ✅(16) Interview_102.md
🎧    Interview_104.md
🎧    Interview_108.md
🎧    Interview_110.md
🎧    Interview_111.md
🎧    Interview_112.md
🎧    Interview_114.md
🎧    Interview_12.md
🎧    Interview_120.md
🎧    Interview_121.md
🎧    Interview_122.md
🎧    Interview_126.md
🎧    Interview_128.md
🎧    Interview_132.md
🎧    Interview_135.md
🎧    Interview_136.md
🎧    Interview_137.md
🎧    Interview_138.md
🎧    Interview_139.md
🎧    Interview_14.md
🎧    Interview_140.md
🎧    Interview_145.md
🎧    Interview_150.md
🎧    Interview_151.md
🎧    Interview_152.md
🎧    Interview_154.md
🎧    Interview_156.md
🎧    Interview_158.md
🎧    Interview_16.md
🎧    Interview_160.md
🎧    Interview_166.md
🎧    Interview_175.md
🎧    Interview_177.md
🎧    Interview_178.md
🎧    Interview_179.md
🎧    Interview_180.md
🎧    Interview_181.md
🎧    Interview_182.md
🎧    Interview_183.md
🎧    Interview_184.md
🎧    Interview_187.md
🎧    Interview_194.md
🎧